In [1]:
using Turing, Distributions
using PyPlot, PyCall
@pyimport matplotlib.cm as cm
@pyimport numpy as np

In [2]:
# Load transition matrix T and observations obs
include(Pkg.dir("Turing")*"/notebooks/data/hmmdemo.data.jl");

In [3]:
K = 5
N = 201
initial = fill(1.0 / K, K)
means = (collect(1.0:K)*2-K)*2

@model hmmdemo begin
    states = tzeros(Int,N)
    # T = TArray{Array{Float64,}}
    
    # Prior over T
    # for i=1:K, @assume T[i,:] ~ Dirichlet(ones(K)./K); end
    
    @assume states[1] ~ Categorical(initial)
    for i = 2:N
        @assume states[i] ~ Categorical(vec(T[states[i-1],:]))
        @observe obs[i] ~ Normal(means[states[i]], 4)
    end
    @predict states
end

hmmdemo (generic function with 1 method)

In [4]:
chain = sample(hmmdemo, PG(100,200));

[PG]: Iter 10 out of 200
[PG]: Iter 20 out of 200
[PG]: Iter 30 out of 200
[PG]: Iter 40 out of 200
[PG]: Iter 50 out of 200
[PG]: Iter 60 out of 200
[PG]: Iter 70 out of 200
[PG]: Iter 80 out of 200
[PG]: Iter 90 out of 200
[PG]: Iter 100 out of 200
[PG]: Iter 110 out of 200
[PG]: Iter 120 out of 200
[PG]: Iter 130 out of 200
[PG]: Iter 140 out of 200
[PG]: Iter 150 out of 200
[PG]: Iter 160 out of 200
[PG]: Iter 170 out of 200
[PG]: Iter 180 out of 200
[PG]: Iter 190 out of 200
[PG]: Iter 200 out of 200


In [5]:
# Print the 1st sample trajectory
chain.value[1].value[:states]'
chain.value[100]

Turing.Sample(0.005,Dict{Symbol,Any}(:states=>[5,5,5,5,3,3,1,2,2,2  …  1,1,1,3,1,1,1,1,1,1]))

In [ ]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
PyPlot.hold(false)
colors = cm.rainbow(np.linspace(0, 1, K))
fig = gcf()
fig[:figsize]=(12,6)

for j=1:5;
for i=1:length(chain.value)
    # Plot trajectories
    plot(1:N, means[chain.value[i].value[:states]], 1:N, obs, "rs")
    
    # Then force the draw
    pause(0.05)
end
end